In [63]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.functions import countDistinct

In [5]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
print(spark.version)

3.4.1


In [7]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/data/actor.csv.

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [27]:
films_df = film_category_df.groupBy("category_id").count()
df_joined = category_df[["category_id", "name"]].join(films_df, on="category_id", how="inner")
df_joined.sort(df_joined["count"].desc()).show()

+-----------+-----------+-----+
|category_id|       name|count|
+-----------+-----------+-----+
|         15|     Sports|   74|
|          9|    Foreign|   73|
|          8|     Family|   69|
|          6|Documentary|   68|
|          2|  Animation|   66|
|          1|     Action|   64|
|         13|        New|   63|
|          7|      Drama|   62|
|         10|      Games|   61|
|         14|     Sci-Fi|   61|
|          3|   Children|   60|
|          5|     Comedy|   58|
|         16|     Travel|   57|
|          4|   Classics|   57|
|         11|     Horror|   56|
|         12|      Music|   51|
+-----------+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [29]:
inventory_df.show()

+------------+-------+--------+-------------------+
|inventory_id|film_id|store_id|        last_update|
+------------+-------+--------+-------------------+
|           1|      1|       1|2022-02-15 10:09:17|
|           2|      1|       1|2022-02-15 10:09:17|
|           3|      1|       1|2022-02-15 10:09:17|
|           4|      1|       1|2022-02-15 10:09:17|
|           5|      1|       2|2022-02-15 10:09:17|
|           6|      1|       2|2022-02-15 10:09:17|
|           7|      1|       2|2022-02-15 10:09:17|
|           8|      1|       2|2022-02-15 10:09:17|
|           9|      2|       2|2022-02-15 10:09:17|
|          10|      2|       2|2022-02-15 10:09:17|
|          11|      2|       2|2022-02-15 10:09:17|
|          12|      3|       2|2022-02-15 10:09:17|
|          13|      3|       2|2022-02-15 10:09:17|
|          14|      3|       2|2022-02-15 10:09:17|
|          15|      3|       2|2022-02-15 10:09:17|
|          16|      4|       1|2022-02-15 10:09:17|
|          1

In [36]:
df_joined = rental_df.join(inventory_df, on="inventory_id", how="inner")
df_joined = df_joined.join(film_actor_df, on="film_id", how="inner")
df_joined = df_joined.join(actor_df, on="actor_id", how="inner")
df_grouped = df_joined.groupBy(["first_name", "last_name"]).count()
df_grouped.sort(df_grouped["count"].desc()).show()

+----------+-----------+-----+
|first_name|  last_name|count|
+----------+-----------+-----+
|     SUSAN|      DAVIS|  825|
|      GINA|  DEGENERES|  753|
|   MATTHEW|     CARREY|  678|
|      MARY|     KEITEL|  674|
|    ANGELA|WITHERSPOON|  654|
|    WALTER|       TORN|  640|
|     HENRY|      BERRY|  612|
|     JAYNE|      NOLTE|  611|
|       VAL|     BOLGER|  605|
|    SANDRA|     KILMER|  604|
|      SEAN|    GUINESS|  599|
|    ANGELA|     HUDSON|  574|
|  SCARLETT|      DAMON|  572|
|      EWAN|    GOODING|  571|
|     KEVIN|    GARLAND|  565|
|    WARREN|      NOLTE|  564|
|   CAMERON|  ZELLWEGER|  560|
|     WOODY|    HOFFMAN|  560|
|      JADA|      RYDER|  560|
|     HELEN|     VOIGHT|  557|
+----------+-----------+-----+
only showing top 20 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [73]:
df_joined = film_category_df.join(category_df, on="category_id", how="inner")
df_joined = df_joined.join(film_df, on="film_id", how="inner")
df_joined = df_joined.join(inventory_df, on="film_id", how="inner")
df_joined = df_joined.join(rental_df, on="inventory_id", how="inner")
df_joined = df_joined.join(payment_df, on="rental_id", how="inner")

df_grouped = df_joined.groupBy("name").sum("amount")
df_grouped.sort(df_grouped["sum(amount)"].desc()).limit(1).show()

+------+-----------------+
|  name|      sum(amount)|
+------+-----------------+
|Sports|5314.209999999843|
+------+-----------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [55]:
df_joined = film_df.join(inventory_df, on="film_id", how="left")
df_joined.filter(df_joined["film_id"].isNull()).show()

+-------+-----+-----------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+-----------+----------------+--------+------------+--------+-----------+
|film_id|title|description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|last_update|special_features|fulltext|inventory_id|store_id|last_update|
+-------+-----+-----------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+-----------+----------------+--------+------------+--------+-----------+
+-------+-----+-----------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+-----------+----------------+--------+------------+--------+-----------+



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [77]:
df_joined = film_actor_df.join(film_category_df, on="film_id", how="inner")
df_joined = df_joined.join(category_df, on="category_id", how="inner")
df_joined.filter(df_joined["name"] == 'Children')
df_grouped = df_joined.groupBy("actor_id").agg(countDistinct("film_id"))
top_actors = df_grouped.sort(df_grouped["count(film_id)"].desc()).limit(3)
top_actors.join(actor_df, on="actor_id", how="inner").select(["first_name", "last_name"]).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|    WALTER|     TORN|
|      GINA|DEGENERES|
|      MARY|   KEITEL|
+----------+---------+



In [78]:
top_actors.show()

+--------+--------------+
|actor_id|count(film_id)|
+--------+--------------+
|     107|            42|
|     102|            41|
|     198|            40|
+--------+--------------+



Stop Spark session:

In [79]:
spark.stop()